Parse CZI file metadata as XML Tree

In [2]:
import xml.etree.ElementTree as ET
import czifile

file_path = 'images/Dox_FRAP_24.czi'

def parse_graphic_annotation_czi(file_path):
    '''
    parse_graphic_annotation_czi(file_path)
    Take in a file path to a czi file
    Uses the package cziFile to read in czi file and then parses metadata xml tree to find the first graphic annotation.
    Returns the data for the annotation's location as a tuple of lists
    '''
    czi2 = czifile.CziFile(file_path)
    # pull out metadata xml tree as a string
    xml_mdata = czi2.metadata()
    # convert to an xml tree object
    root = ET.fromstring(xml_mdata)
    # get the first location of graphic annotations and the list of elements
    ann_layer = root.find(".//*[@Name='Annotation']")
    elems = ann_layer.find("Elements")

    # check if the annotation is a circle or an ellipse and locate annotation
    # This will favor whichever annotation is listed last, which should be ok since their is only one
    ellipse = True
    for child in elems.findall("*"):
        if child.tag == "Circle":
            ellipse = False
            elem = child
        elif child.tag == "Ellipse":
            ellipse = True
            elem = child

    # organize location information into a list
    geom_data = []
    geom_name = [] # stores the names for each number
    # pull out location information from element
    elem_geom = elem.find("Geometry")
    count = 0
    for child in elem_geom.findall("*"):
        # if circle, grab first four children
        if (ellipse and count < 4):
            geom_data.append(float(child.text))
            geom_name.append(child.tag)
        elif (count < 3): # if it is ellipse, grab first four children
            geom_data.append(float(child.text))
            geom_name.append(child.tag)
        count+=1
    # return a tuple of lists
    return (geom_data, geom_name)

parsed_annotations = parse_graphic_annotation_czi(file_path=file_path)
print(parsed_annotations[0])

[274.93751989862915, 161.8981397535626, 35.82531369510575, 40.087299808981626]
